In [0]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

from google.colab import drive
drive.mount('/content/drive')

class MLP(nn.Module):
  def __init__(self, n_units, init_scale=1.0):
    super(MLP, self).__init__()

    self._n_units = copy.copy(n_units)
    self._layers = []
    for i in range(1, len(n_units)):
      layer = nn.Linear(n_units[i-1], n_units[i], bias=False)
      variance = math.sqrt(2.0 / (n_units[i-1] + n_units[i]))
      layer.weight.data.normal_(0.0, init_scale * variance)
      self._layers.append(layer)

      name = 'fc%d' % i
      if i == len(n_units) - 1:
        name = 'fc'  # the prediction layer is just called fc
      self.add_module(name, layer)


  def forward(self, x):
    x = x.view(-1, self._n_units[0])
    out = self._layers[0](x)
    for layer in self._layers[1:]:
      out = F.relu(out)
      out = layer(out)
    return out



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/'My Drive'

/content/drive/My Drive


In [0]:
def get_data(seed):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)
  np.random.seed(seed)

  probability_of_random = 1.0
  labels = np.array(train_dataset.targets) 
  mask = np.random.rand(len(labels)) <= probability_of_random #create mask of length labels, where entries drawn from [0,1].
  rnd_labels = np.random.choice(10, mask.sum())               #create random labels 1-10 of length of mask
  labels[mask] = rnd_labels
  labels = [int(x) for x in labels]
  train_dataset.targets = labels                              #assign new random labels to dataset
  
  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=128,
                          num_workers=4,
                          shuffle=True)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=128,
                         num_workers=4,
                         shuffle=False)
  return train_loader, test_loader

In [0]:
def compute_acc(model, data_loader):
    correct_pred, num_examples = 0, 0
    model.eval()
    for i, (features, targets) in enumerate(data_loader):
        probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        assert predicted_labels.size() == targets.size()
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [0]:
weights = []
weights2 = []
models = []

seeds = 10

train_loader, test_loader = get_data(0)

for i in range(seeds):
  model = torch.load('MLP_fixedrandom_iter' + str(i), map_location=lambda storage, loc: storage)
  
  models.append(model)
  weights.append(model.fc1.weight)
  weights2.append(model.fc.weight)
  #print(model.fc1.weight)
  #print(model.fc.weight)

  train_acc = compute_acc(model, train_loader)        
  print(f'train ACC: {train_acc:.2f}, seed: {i}')
        
  test_acc = compute_acc(model, test_loader)        
  print(f'Test ACC: {test_acc:.2f}')

Files already downloaded and verified
Files already downloaded and verified
train ACC: 99.28, seed: 0
Test ACC: 9.79
train ACC: 99.23, seed: 1
Test ACC: 9.99
train ACC: 99.32, seed: 2
Test ACC: 9.75
train ACC: 99.29, seed: 3
Test ACC: 9.49
train ACC: 99.23, seed: 4
Test ACC: 9.49
train ACC: 99.31, seed: 5
Test ACC: 10.07
train ACC: 99.35, seed: 6
Test ACC: 9.99
train ACC: 99.30, seed: 7
Test ACC: 9.30
train ACC: 99.28, seed: 8
Test ACC: 10.35
train ACC: 99.27, seed: 9
Test ACC: 9.81


In [0]:
norms = []
for i in range(seeds):
  for j in range(seeds):
    if i != j:
      cross_corr = weights[i] @ weights[j].t()
      norms.append(torch.norm(cross_corr, p='fro'))


In [0]:
print(norms)

[tensor(159.4062, grad_fn=<NormBackward0>), tensor(159.1116, grad_fn=<NormBackward0>), tensor(159.3288, grad_fn=<NormBackward0>), tensor(159.1003, grad_fn=<NormBackward0>), tensor(159.1897, grad_fn=<NormBackward0>), tensor(159.7686, grad_fn=<NormBackward0>), tensor(158.6126, grad_fn=<NormBackward0>), tensor(158.4450, grad_fn=<NormBackward0>), tensor(159.0246, grad_fn=<NormBackward0>), tensor(159.4076, grad_fn=<NormBackward0>), tensor(159.4506, grad_fn=<NormBackward0>), tensor(159.5148, grad_fn=<NormBackward0>), tensor(159.7205, grad_fn=<NormBackward0>), tensor(159.4501, grad_fn=<NormBackward0>), tensor(160.0533, grad_fn=<NormBackward0>), tensor(159.2927, grad_fn=<NormBackward0>), tensor(159.1921, grad_fn=<NormBackward0>), tensor(159.3202, grad_fn=<NormBackward0>), tensor(159.1111, grad_fn=<NormBackward0>), tensor(159.4507, grad_fn=<NormBackward0>), tensor(159.3022, grad_fn=<NormBackward0>), tensor(159.3078, grad_fn=<NormBackward0>), tensor(159.3461, grad_fn=<NormBackward0>), tensor(159

In [0]:
!pip install weightwatcher

  Created wheel for powerlaw: filename=powerlaw-1.4.6-cp36-none-any.whl size=24787 sha256=060974ded0579052abd1b60156e9ddae5f033cbb64f938edff7911743f643da6
  Stored in directory: /root/.cache/pip/wheels/e0/27/02/08d0e2865072bfd8d7c655e290521e3feca3fc22f1ac460601
Successfully built powerlaw


In [0]:
 import weightwatcher as ww

Using TensorFlow backend.


In [0]:
summaries = []

for i in range(seeds):
  watcher = ww.WeightWatcher(model=models[i])
  results = watcher.analyze(alphas=True, spectralnorms=True, mp_fit=True)
  t = watcher.get_summary()
  summaries.append(t)


2020-04-08 17:28:58,721 INFO 

python      version 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
numpy       version 1.18.2
tensforflow version 2.2.0-rc2
keras       version 2.2.5
2020-04-08 17:28:58,723 INFO Analyzing model
/usr/local/lib/python3.6/dist-packages/powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/usr/local/lib/python3.6/dist-packages/powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
2020-04-08 17:29:02,305 INFO ### Printing results ###
2020-04-08 17:29:02,453 INFO Check: min: 0.0482854029751311, max: 0.1748633958621577, avg: 0.1115743994186444
2020-04-08 17:29:02,454 INFO Check compound: min: 0.0482854029751311, max: 0.1748633958621577, avg: 0.1115743994186444
2020-04-08 17:29:02,456 INFO CheckTF: min: False, max: False, avg: 0.0
2020-04-08 17:29:02,457 INFO CheckTF compound: min: 0.0, max: 0.0, avg: 0.0
2020-04-08 17:29:02,459 

In [0]:
for i in range(len(summaries)):
  print("Random Labels, iteration", i)
  print('alpha', summaries[i]['alpha'])
  print('spectral norm', summaries[i]['spectralnorm'])
  print('norm', summaries[i]['norm'])

Random Labels, iteration 0
alpha 7.138388869186606
spectral norm 23.215866
norm 42.819977
Random Labels, iteration 1
alpha 6.044632746348675
spectral norm 23.60924
norm 42.855206
Random Labels, iteration 2
alpha 7.149940989870606
spectral norm 23.149578
norm 42.843098
Random Labels, iteration 3
alpha 6.741626273433219
spectral norm 23.603287
norm 42.800556
Random Labels, iteration 4
alpha 5.56939585020806
spectral norm 23.546207
norm 42.86681
Random Labels, iteration 5
alpha 6.715481432421792
spectral norm 23.351536
norm 42.829205
Random Labels, iteration 6
alpha 5.761497790193734
spectral norm 23.636631
norm 42.8218
Random Labels, iteration 7
alpha 6.073904534683159
spectral norm 22.89254
norm 42.85055
Random Labels, iteration 8
alpha 7.0008417138624015
spectral norm 22.936443
norm 42.81159
Random Labels, iteration 9
alpha 6.441120356287457
spectral norm 23.44408
norm 42.84715
